# Spectral Clustering

## Important imports

In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from scipy.spatial import distance_matrix as distanceMatrix
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import rbf_kernel

## Data and Preprocessing

In [6]:
def loadData(path):
    data = pd.read_csv(path)
    x = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return x, y

def splitData(x, y, train_size):
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0, stratify=y)
    return x_train, x_test, y_train, y_test

def preprocessData(x):
    ct = ColumnTransformer( [('one_hot_encoder', OneHotEncoder(), [1, 2, 3])], remainder='passthrough' )
    ct = ct.fit(x)
    x = pd.DataFrame(ct.transform(x))
    # Drop the first, second and third columns
    x = x.drop([1, 2, 3], axis=1)
    return x

## Algorithm

In [7]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
def constructWeightMatrix(data, gamma = 10):
    K=rbf_kernel(data, gamma=gamma)#the kernel matrix
    K_symmetric=(K+K.T)/2
    #to make it symmetric
    n=25#as the number of features are 122
    A=kneighbors_graph(K_symmetric, n_neighbors=n, mode='connectivity')#creating the connectivity graph
    A=A.toarray()#toconvert it from sparse to dense matrix to be used in the spectral clustering
    A_symmetric=(A+A.T)/2#to make it symmetric
    A_symmetric[A_symmetric > 0] = 1 
    return A_symmetric

def clusteringUsingNormalizedCut(data, k, gamma = 10, y = None):
    # Constructing the weight matrix
    weight_matrix = constructWeightMatrix(data, gamma=gamma)

    # Computing the degree matrix
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))

    # Computing the laplacian matrix
    laplacian_matrix = degree_matrix - weight_matrix

    # Computing the eigenvalues and eigenvectors of the laplacian matrix
    eigenvalues, eigenvectors = np.linalg.eig(np.linalg.inv(degree_matrix).dot(laplacian_matrix))
    
    # Taking the only first k eigenvectors
    eigenvectors = eigenvectors[:, :k]

    # Taking only the real part of the eigenvectors
    eigenvectors = np.real(eigenvectors)

    # Convert the eigenvectors to pd dataframe
    eigenvectors = pd.DataFrame(eigenvectors)
    
    # Normalizing the eigenvectors
    eigenvectors = eigenvectors.apply(lambda x: (x - np.mean(x)) / np.std(x), axis=0)
    
    print(eigenvectors.shape)
    # Applying k-means on the eigenvectors
    kmeans = KMeans(n_clusters=k, random_state=0).fit(eigenvectors)

    # Getting the labels of the points
    labels = kmeans.labels_ 

    # Create a list of clusters of size k
    cluster = [[] for i in range(k)]

    # Assign each point to its cluster
    for i in range(len(labels)):
        cluster[labels[i]].append(i)
    
    # Create array name clusterLabels to store the labels of the clusters based on the most frequent label in the cluster in y
    clusterLabels = []
    for i in range (k):
        counterLabels = {}
        for j in range(len(cluster[i])):
            counterLabels[y[cluster[i][j]]] = counterLabels.get(y[cluster[i][j]], 0) + 1
        # Assign the most frequent label to the cluster
        maxLabel = max(counterLabels, key=counterLabels.get)
        clusterLabels.append(maxLabel)
    
    pointsLabels = []
    for i in range(len(labels)):
        pointsLabels.append(clusterLabels[labels[i]])

    return cluster, pointsLabels

## Evalaution

### Load the data

In [8]:
# Loading the data
x, y = loadData('archive/kddcup.data.corrected')

### Preprocessing the Data

In [9]:
x = preprocessData(x)
print("The shape of the training data is: ", x.shape)

The shape of the training data is:  (4898430, 119)


### Split the data

In [10]:
# Splitting the data into train and test
x_train, x_test, y_train, y_test = splitData(x, y, train_size= 0.00025)

### Evaluating

In [11]:
clusters, y_pred = clusteringUsingNormalizedCut(x_train, 23, gamma=0.1, y=y_train)
y_pred = np.array(y_pred)

(1224, 23)


c:\python\python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [12]:
print("Weight")
print("Precision: ", precision_score(y_train, y_pred, average='weighted'))
print("Recall: ", recall_score(y_train, y_pred, average='weighted'))
print("F1 score: ", f1_score(y_train, y_pred, average='weighted'))
print("Accuracy: ", accuracy_score(y_train, y_pred))

print("-" * 100)
print("Macro")
print("Precision: ", precision_score(y_train, y_pred, average='macro'))
print("Recall: ", recall_score(y_train, y_pred, average='macro'))
print("F1 score: ", f1_score(y_train, y_pred, average='macro'))
print("Accuracy: ", accuracy_score(y_train, y_pred))

print("-" * 100)

print(classification_report(y_train, y_pred))

Weight
Precision:  0.9129535534219594
Recall:  0.875
F1 score:  0.8710864370111667
Accuracy:  0.875
----------------------------------------------------------------------------------------------------
Macro
Precision:  0.37250025551051
Recall:  0.35974414995973875
F1 score:  0.35063705278118545
Accuracy:  0.875
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

    ipsweep.       0.00      0.00      0.00         3
    neptune.       0.99      0.55      0.71       268
       nmap.       0.00      0.00      0.00         1
     normal.       0.61      1.00      0.76       243
  portsweep.       0.00      0.00      0.00         3
      satan.       0.00      0.00      0.00         4
      smurf.       1.00      0.97      0.98       702

    accuracy                           0.88      1224
   macro avg       0.37      0.36      0.35      1224
weighted avg       0.91      0.88      0.87 

c:\python\python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python\python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python\python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python\python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWa